# Содержание
- [О Guidance](../../../../code/01.Introduce)
- [Настройка](../../../../code/01.Introduce)
- [Независимая генерация](../../../../code/01.Introduce)
- [Говоря за Phi 3](../../../../code/01.Introduce)
- [Regex](../../../../code/01.Introduce)
- [Выбор](../../../../code/01.Introduce)
- [Цепочка рассуждений](../../../../code/01.Introduce)
- [Генерация JSON](../../../../code/01.Introduce)
- [Генерация HTML](../../../../code/01.Introduce)


# О Guidance
Guidance — это проверенная временем библиотека с открытым исходным кодом для Python, предназначенная для управления выводами любых языковых моделей (LM). С помощью одного вызова API вы можете задать (на Python) точные программные ограничения, которым должна следовать модель, и получить структурированный вывод в формате JSON, Python, HTML, SQL или любой другой структуре, необходимой для вашего сценария использования.

Guidance отличается от традиционных методов подсказок. Он применяет ограничения, направляя модель токен за токен на уровне инференса, что позволяет получать более качественные результаты и снижать затраты и задержки на 30–50% при использовании в сценариях с высокой степенью структурированности.

Чтобы узнать больше о Guidance, посетите [публичный репозиторий на GitHub](https://github.com/guidance-ai/guidance) или посмотрите [сессию Guidance Breakout](https://www.youtube.com/watch?v=qXMNPVVlCMs) на Microsoft Build.


# Настройка
1. Установите Guidance с помощью команды `pip install guidance --pre`
2. Разверните мини-версию Phi 3.5 в Azure, перейдя по ссылке https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml и нажав кнопку "Deploy"
3. Сохраните API-ключ вашего конечного пункта в переменной окружения с именем `AZURE_PHI3_KEY`, а URL — в переменной окружения с именем `AZURE_PHI3_URL`


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# Неконтролируемая генерация
Текст может быть сгенерирован без каких-либо ограничений с помощью функции `gen()`. Это то же самое, что использование модели без Guidance.

## Форматирование чата
Как и многие модели чата, Phi-3 ожидает сообщения между пользователем и ассистентом в определенном формате. Guidance поддерживает шаблон чата Phi-3 и будет управлять форматированием чата за вас. Чтобы создать очереди чата, поместите каждую часть разговора в блок `with user()` или `with assistant()`. Блок `with system()` может быть использован для установки системного сообщения.


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## Экономия токенов
В строго структурированных сценариях Guidance может пропускать токены и генерировать только необходимые, что улучшает производительность, повышает эффективность и снижает затраты на API. Сгенерированные токены отображаются в этом блокноте с выделенным фоном. Принудительные токены отображаются без выделения и стоят столько же, сколько входные токены, которые оцениваются примерно в одну треть стоимости выходных токенов.

*Примечание:* В первом примере с неограниченной генерацией не удалось принудительно задать токены, так как мы не предоставили никаких ограничений.


# Говорим за Phi 3  
С помощью Guidance вы можете легко добавлять текст в ответы модели. Это может быть полезно, если вы хотите направить вывод модели в определённое русло.


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# Ограничение с помощью регулярных выражений  
В предыдущем примере Phi 3 дал дополнительные объяснения после ответа на вопрос с `Canberra`. Чтобы ограничить вывод модели до строго одного слова, можно использовать регулярное выражение.


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

# Выбор из нескольких вариантов
Когда известны некоторые возможные варианты, вы можете использовать функцию `select()`, чтобы модель выбрала из списка опций.


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

С помощью `select()` был сгенерирован только токен `Can`. Поскольку `Canberra` является единственным вариантом, который может завершить ответ, оставшиеся токены были принудительно добавлены.


# Цепочка рассуждений
Цепочка рассуждений — это техника, которая помогает улучшить качество вывода модели, побуждая её решать задачу шаг за шагом. Обычно для получения окончательного ответа требуется несколько этапов подсказок. Сначала нужно дать модели инструкцию думать пошагово. Затем снова запросить модель, чтобы она предоставила окончательный ответ. При использовании стандартных API для чат-инференса это требует двух вызовов API, и сгенерированная моделью «цепочка рассуждений» оплачивается дважды — сначала как выходные токены, когда модель её создала, а затем как входные токены для второго вызова. С помощью Guidance весь многоэтапный процесс обрабатывается и оплачивается в рамках одного вызова API, что снижает затраты и задержки.


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# Генерация JSON
Guidance можно использовать для гарантии генерации JSON, соответствующего JSON-схеме или модели pydantic, например, схеме профиля пользователя, показанной здесь.


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## Генерация HTML

Guidance также можно использовать для создания кода и соблюдения синтаксических требований программного языка. В этом разделе мы создадим небольшую программу Guidance для написания простых HTML-страниц.

Мы разобьем веб-страницу на более мелкие секции, каждая из которых будет иметь свою функцию Guidance. Затем эти секции будут объединены в нашей финальной функции для создания HTML-страницы. 
После этого мы запустим эту функцию с использованием модели, поддерживающей Guidance, в Azure AI.

*Примечание:* Это не будет полнофункциональный генератор HTML; цель состоит в том, чтобы показать, как можно создавать структурированный вывод для ваших индивидуальных потребностей.

Начнем с импорта необходимых компонентов из Guidance:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

HTML-страницы имеют четкую структуру, и мы будем «принуждать» эти части страницы с помощью Guidance.  
Когда мы явно запрашиваем текст у модели, необходимо убедиться, что он не содержит ничего, что могло бы быть тегом, то есть мы должны исключить символы '<' и '>'.


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

Затем мы можем использовать эту функцию для генерации текста внутри произвольного HTML-тега:


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

Теперь давайте создадим заголовок страницы.  
В рамках этого нам нужно сгенерировать заголовок страницы:


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

Тело HTML-страницы будет заполнено заголовками и абзацами.  
Мы можем определить функцию для выполнения каждой задачи:


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

Теперь функция для определения основного содержимого самого HTML.  
Она использует `select()` с параметром `recurse=True` для создания нескольких заголовков и абзацев:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

Далее мы переходим к функции, которая генерирует полный HTML-страницу.  
Мы добавляем начальный тег HTML, затем создаем заголовок, затем тело, и в конце добавляем завершающий тег HTML:


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

Мы предоставляем удобный интерфейс, который позволит нам:
- Устанавливать температуру генерации
- Захватывать сгенерированную страницу из объекта Model


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

Мы можем затем записать вывод в файл:


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

И [посмотрите результат](../../../../code/01.Introduce/sample_page.html).



---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Хотя мы стремимся к точности, пожалуйста, учитывайте, что автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его родном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
